基于plotly的简单数据探索

In [86]:
import numpy as np
import pandas as pd

import plotly as py
import plotly.graph_objs as go

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [46]:
train_offline_data_type = {'User_id': np.int64, 'Merchant_id': np.int64, 'Coupon_id': object,
              'Discount_rate': object, 'Distance' :object, 'Date_received': object,
              'Date': object}
test_offline_data_type = {'User_id': np.int64, 'Merchant_id': np.int64, 'Coupon_id': object,
              'Discount_rate': object, 'Distance' :object, 'Date_received': object}
train_online_data_type = {'User_id': np.int64, 'Merchant_id': np.int64, 'Action': np.int64,
              'Coupon_id': object, 'Discount_rate': object, 'Distance' :object,
              'Date_received': object}
              
test_offline = pd.read_csv('/content/drive/My Drive/Case/O2O Coupon Use Prediction/data/ccf_offline_stage1_test_revised.csv', dtype=test_offline_data_type)
train_offline = pd.read_csv('/content/drive/My Drive/Case/O2O Coupon Use Prediction/data/ccf_offline_stage1_train.csv', dtype=train_offline_data_type)
train_online = pd.read_csv('/content/drive/My Drive/Case/O2O Coupon Use Prediction/data/ccf_online_stage1_train.csv', dtype=train_online_data_type)
submission = pd.read_csv('/content/drive/My Drive/Case/O2O Coupon Use Prediction/data/sample_submission.csv')

In [47]:
print('test_offline dataset has {} rows and {} columns.'.format(test_offline.shape[0],test_offline.shape[1]))
print('train_offline dataset has {} rows and {} columns.'.format(train_offline.shape[0],train_offline.shape[1]))
print('train_online dataset has {} rows and {} columns.'.format(train_online.shape[0],train_online.shape[1]))
print('submission dataset has {} rows and {} columns.'.format(submission.shape[0],submission.shape[1]))

test_offline dataset has 113640 rows and 6 columns.
train_offline dataset has 1754884 rows and 7 columns.
train_online dataset has 11429826 rows and 7 columns.
submission dataset has 13 rows and 4 columns.


辅助字段

In [104]:
train_offline['month'] = train_offline['Date'].apply(lambda x: str(x)[4:6])
train_offline['Received_month'] = train_offline['Date_received'].apply(lambda x: str(x)[4:6])
train_offline['Received_But_No_Use'] = ((train_offline['Date'].isna())&(~train_offline['Coupon_id'].isna()))*1
train_offline['Not_Received'] = ((~train_offline['Date'].isna())&(train_offline['Coupon_id'].isna()))*1
train_offline['Received_And_Use'] = ((~train_offline['Date'].isna())&(~train_offline['Coupon_id'].isna()))*1
train_online['month'] = train_online['Date'].apply(lambda x: str(x)[4:6])
train_online['Received_month'] = train_online['Date_received'].apply(lambda x: str(x)[4:6])
train_online['Received_But_No_Use'] = ((train_online['Date'].isna())&(~train_online['Coupon_id'].isna()))*1
train_online['Not_Received'] = ((~train_online['Date'].isna())&(train_online['Coupon_id'].isna()))*1
train_online['Received_And_Use'] = ((~train_online['Date'].isna())&(~train_online['Coupon_id'].isna()))*1

## 优惠券消费占比 by月份

线下

In [94]:
temp = pd.merge(train_offline[train_offline['month']!=""].groupby('month')['Not_Received'].sum(),
     train_offline[train_offline['month']!=""].groupby('month')['Received_And_Use'].sum(),
     on='month')
trace_1 = go.Bar(
        x=temp.index,
        y=temp['Not_Received'],
        name="普通消费")
trace_2 = go.Bar(
        x=temp.index,
        y=temp['Received_And_Use'],
        name="优惠券消费")
trace = [trace_1, trace_2]
 
layout = go.Layout(
        title='2016年上半年线下每月优惠券消费占比',
        barmode='stack')
figure = go.Figure(data=trace, layout=layout)
#py.offline.plot(figure, filename='fig.html')
figure

线上

In [93]:
temp = pd.merge(train_online[train_online['month']!=""].groupby('month')['Not_Received'].sum(),
     train_online[train_online['month']!=""].groupby('month')['Received_And_Use'].sum(),
     on='month')
trace_1 = go.Bar(
        x=temp.index,
        y=temp['Not_Received'],
        name="普通消费")
trace_2 = go.Bar(
        x=temp.index,
        y=temp['Received_And_Use'],
        name="优惠券消费")
trace = [trace_1, trace_2]
 
layout = go.Layout(
        title='2016年上半年线上每月优惠券消费占比',
        barmode='stack')
figure = go.Figure(data=trace, layout=layout)
#py.offline.plot(figure, filename='fig.html')
figure

## 用户优惠券使用情况

线上

In [117]:
temp = pd.merge(train_online[train_online['Received_And_Use']==1].groupby('month')['User_id'].aggregate('nunique').reset_index(),
     train_online[train_online['Received_month']!=""].groupby('Received_month')['User_id'].aggregate('nunique').reset_index(),
     left_on='month', right_on='Received_month')

trace_1 = go.Scatter(
        x=temp['month'],
        y=temp['User_id_x'],
        name="使用了消费券的用户数")
trace_2 = go.Scatter(
        x=temp['month'],
        y=temp['User_id_y'],
        name="领取了消费券的用户数")
trace = [trace_1, trace_2]
 
layout = go.Layout(
        title='2016年上半年线上每月优惠券用户使用情况')
figure = go.Figure(data=trace, layout=layout)
#py.offline.plot(figure, filename='fig.html')
figure


线下

In [119]:
temp = pd.merge(train_offline[train_offline['Received_And_Use']==1].groupby('month')['User_id'].aggregate('nunique').reset_index(),
     train_offline[train_offline['Received_month']!=""].groupby('Received_month')['User_id'].aggregate('nunique').reset_index(),
     left_on='month', right_on='Received_month')

trace_1 = go.Scatter(
        x=temp['month'],
        y=temp['User_id_x'],
        name="使用了消费券的用户数")
trace_2 = go.Scatter(
        x=temp['month'],
        y=temp['User_id_y'],
        name="领取了消费券的用户数")
trace = [trace_1, trace_2]
 
layout = go.Layout(
        title='2016年上半年线下每月优惠券用户使用情况')
figure = go.Figure(data=trace, layout=layout)
#py.offline.plot(figure, filename='fig.html')
figure

95%的人只会产生1到3次使用优惠券购买，只有少部分人会在一个月内较多次使用优惠券的购买。

In [143]:
print(train_offline[train_offline['Received_And_Use']==1].groupby(['month','User_id'])['User_id'].aggregate('count').groupby('month').mean())
print(train_offline[train_offline['Received_And_Use']==1].groupby(['month','User_id'])['User_id'].aggregate('count').groupby('month').median())
print(train_offline[train_offline['Received_And_Use']==1].groupby(['month','User_id'])['User_id'].aggregate('count').groupby('month').quantile(q=0.95))
print(train_offline[train_offline['Received_And_Use']==1].groupby(['month','User_id'])['User_id'].aggregate('count').groupby('month').quantile(q=0.99))
print(train_offline[train_offline['Received_And_Use']==1].groupby(['month','User_id'])['User_id'].aggregate('count').groupby('month').max())

month
01    1.327181
02    1.450138
03    1.362362
04    1.289710
05    1.431802
06    1.300635
Name: User_id, dtype: float64
month
01    1
02    1
03    1
04    1
05    1
06    1
Name: User_id, dtype: int64
month
01    3.0
02    3.0
03    2.0
04    2.0
05    3.0
06    3.0
Name: User_id, dtype: float64
month
01    6.0
02    7.0
03    4.0
04    5.0
05    7.0
06    6.0
Name: User_id, dtype: float64
month
01    17
02    54
03    27
04    41
05    73
06    19
Name: User_id, dtype: int64


## 商户优惠券发行情况

线上

In [126]:
temp = pd.merge(train_online[train_online['Received_And_Use']==1].groupby('month')['Merchant_id'].aggregate('nunique').reset_index(),
     train_online[train_online['Received_month']!=""].groupby('Received_month')['Merchant_id'].aggregate('nunique').reset_index(),
     left_on='month', right_on='Received_month')

trace_1 = go.Scatter(
        x=temp['month'],
        y=temp['Merchant_id_x'],
        name="消费券被使用过的商家数")
trace_2 = go.Scatter(
        x=temp['month'],
        y=temp['Merchant_id_y'],
        name="发行过消费券的商家数")
trace = [trace_1, trace_2]
 
layout = go.Layout(
        title='2016年上半年线上每月优惠券商家发行情况')
figure = go.Figure(data=trace, layout=layout)
#py.offline.plot(figure, filename='fig.html')
figure


线下

In [127]:
temp = pd.merge(train_offline[train_offline['Received_And_Use']==1].groupby('month')['Merchant_id'].aggregate('nunique').reset_index(),
     train_offline[train_offline['Received_month']!=""].groupby('Received_month')['Merchant_id'].aggregate('nunique').reset_index(),
     left_on='month', right_on='Received_month')

trace_1 = go.Scatter(
        x=temp['month'],
        y=temp['Merchant_id_x'],
        name="消费券被使用过的商家数")
trace_2 = go.Scatter(
        x=temp['month'],
        y=temp['Merchant_id_y'],
        name="发行过消费券的商家数")
trace = [trace_1, trace_2]
 
layout = go.Layout(
        title='2016年上半年线下每月优惠券商家发行情况')
figure = go.Figure(data=trace, layout=layout)
#py.offline.plot(figure, filename='fig.html')
figure


商家优惠券消费的情况存在明显的20/80现象，中位数为2次消费，而均值为十余次消费，这说明右偏十分明显。

In [142]:
print(train_offline[train_offline['Received_And_Use']==1].groupby(['month','Merchant_id'])['Merchant_id'].aggregate('count').groupby('month').mean())
print(train_offline[train_offline['Received_And_Use']==1].groupby(['month','Merchant_id'])['Merchant_id'].aggregate('count').groupby('month').median())
print(train_offline[train_offline['Received_And_Use']==1].groupby(['month','Merchant_id'])['Merchant_id'].aggregate('count').groupby('month').quantile(q=0.95))
print(train_offline[train_offline['Received_And_Use']==1].groupby(['month','Merchant_id'])['Merchant_id'].aggregate('count').groupby('month').quantile(q=0.99))
print(train_offline[train_offline['Received_And_Use']==1].groupby(['month','Merchant_id'])['Merchant_id'].aggregate('count').groupby('month').max())

month
01    18.138138
02    43.006536
03    18.425185
04     7.210700
05     7.983752
06     6.393048
Name: Merchant_id, dtype: float64
month
01    2
02    2
03    2
04    2
05    2
06    2
Name: Merchant_id, dtype: int64
month
01    37.60
02    94.75
03    65.60
04    21.30
05    26.00
06    18.00
Name: Merchant_id, dtype: float64
month
01    503.80
02    861.00
03    266.12
04    117.58
05    126.16
06     73.57
Name: Merchant_id, dtype: float64
month
01     805
02    4848
03    1858
04     539
05     691
06    1595
Name: Merchant_id, dtype: int64


# 优惠券类型

辅助字段

In [180]:
train_offline['Discount_type'] = '-1'
train_offline.loc[train_offline['Discount_rate'].isna(),'Discount_rate'] = '-1'
train_offline.loc[train_offline['Discount_rate'].str.contains(':'),'Discount_type'] = 'ManJian'
train_offline.loc[train_offline['Discount_rate'].str.contains('\.'),'Discount_type'] = 'ZheKou'
train_online['Discount_type'] = '-1'
train_online.loc[train_online['Discount_rate'].isna(),'Discount_rate'] = '-1'
train_online.loc[train_online['Discount_rate'].str.contains(':'),'Discount_type'] = 'ManJian'
train_online.loc[train_online['Discount_rate'].str.contains('\.'),'Discount_type'] = 'ZheKou'
train_online.loc[train_online['Discount_rate'].str.contains('fixed'),'Discount_type'] = 'Fixed'

## 优惠券领取情况

线上

In [205]:
temp = pd.merge(train_online[train_online['Discount_type'] == 'ManJian'].groupby('Received_month')['Discount_type'].count(),
         train_online[train_online['Discount_type'] == 'Fixed'].groupby('Received_month')['Discount_type'].count(),
     on='Received_month')
trace_1 = go.Bar(
        x=temp.index,
        y=temp['Discount_type_x'],
        name="满减优惠券领取数")
trace_2 = go.Bar(
        x=temp.index,
        y=temp['Discount_type_y'],
        name="低价限时优惠优惠券领取数")
trace = [trace_1, trace_2]
 
layout = go.Layout(
        title='2016年上半年线上每月优惠券领取情况',
        barmode='stack')
figure = go.Figure(data=trace, layout=layout)
#py.offline.plot(figure, filename='fig.html')
figure

线下

In [206]:
temp = pd.merge(train_offline[train_offline['Discount_type'] == 'ManJian'].groupby('Received_month')['Discount_type'].count(),
         train_offline[train_offline['Discount_type'] == 'ZheKou'].groupby('Received_month')['Discount_type'].count(),
     on='Received_month')
trace_1 = go.Bar(
        x=temp.index,
        y=temp['Discount_type_x'],
        name="满减优惠券领取数")
trace_2 = go.Bar(
        x=temp.index,
        y=temp['Discount_type_y'],
        name="折扣优惠券领取数")
trace = [trace_1, trace_2]
 
layout = go.Layout(
        title='2016年上半年线下每月优惠券领取情况',
        barmode='stack')
figure = go.Figure(data=trace, layout=layout)
#py.offline.plot(figure, filename='fig.html')
figure

## 优惠券使用情况

线上

In [197]:
temp = pd.merge(train_online[(train_online['Discount_type'] == 'ManJian')&(~train_online['Date'].isna())].groupby('month')['Date'].count(),
         train_online[(train_online['Discount_type'] == 'Fixed')&(~train_online['Date'].isna())].groupby('month')['Date'].count(),
     on='month')
trace_1 = go.Bar(
        x=temp.index,
        y=temp['Date_x'],
        name="满减优惠券使用数")
trace_2 = go.Bar(
        x=temp.index,
        y=temp['Date_y'],
        name="低价限时优惠优惠券使用数")
trace = [trace_1, trace_2]
 
layout = go.Layout(
        title='2016年上半年线上每月优惠券使用情况',
        barmode='stack')
figure = go.Figure(data=trace, layout=layout)
#py.offline.plot(figure, filename='fig.html')
figure

线下

In [203]:
temp = pd.merge([(train_train_offlineoffline['Discount_type'] == 'ManJian')&(~train_offline['Date'].isna())].groupby('month')['Date'].count(),
         train_offline[(train_offline['Discount_type'] == 'ZheKou')&(~train_offline['Date'].isna())].groupby('month')['Date'].count(),
     on='month')
trace_1 = go.Bar(
        x=temp.index,
        y=temp['Date_x'],
        name="满减优惠券使用数")
trace_2 = go.Bar(
        x=temp.index,
        y=temp['Date_y'],
        name="折扣优惠优惠券使用数")
trace = [trace_1, trace_2]
 
layout = go.Layout(
        title='2016年上半年线下每月优惠券使用情况',
        barmode='stack')
figure = go.Figure(data=trace, layout=layout)
#py.offline.plot(figure, filename='fig.html')
figure

打Label

In [207]:
def label(row):
    if row['Date_received'] == 'null':
        return -1
    if row['Date'] != 'null':
        td = pd.to_datetime(row['Date'], format='%Y%m%d') -  pd.to_datetime(row['Date_received'], format='%Y%m%d')
        if td <= pd.Timedelta(15, 'D'):
            return 1
    return 0
train_offline['label'] = train_offline.apply(label, axis = 1)
train_online['label'] = train_online.apply(label, axis = 1)

## 不同优惠券类型的十五天内的使用率

In [223]:
temp = pd.merge(
    pd.Series(name='ManJianUseRate',data=train_offline[(train_offline['Discount_type']=='ManJian')&(train_offline['label']==1)].groupby('month')['label'].count()/train_offline[(train_offline['Discount_type']=='ManJian')].groupby('Received_month')['Discount_type'].count()),
    pd.Series(name='ZheKouUseRate',data=train_offline[(train_offline['Discount_type']=='ZheKou')&(train_offline['label']==1)].groupby('month')['label'].count()/train_offline[(train_offline['Discount_type']=='ZheKou')].groupby('Received_month')['Discount_type'].count()),
    on='month')


trace_1 = go.Scatter(
        x=temp.index,
        y=temp['ManJianUseRate'],
        name="满减优惠券被使用率")
trace_2 = go.Scatter(
        x=temp.index,
        y=temp['ZheKouUseRate'],
        name="折扣优惠券被使用率")
trace = [trace_1, trace_2]
 
layout = go.Layout(
        title='2016年上半年线下每月优惠券使用率')
figure = go.Figure(data=trace, layout=layout)
#py.offline.plot(figure, filename='fig.html')
figure


## 不同距离的优惠券情况如何

In [252]:
temp = train_offline[~train_offline['Date_received'].isna()].groupby('Distance')['Distance'].count()
temp.index = temp.index.astype(int)
temp = temp.sort_index()

trace = go.Scatter(
        x=temp.index,
        y=temp.values)

layout = go.Layout(
        title='2016年上半年不同距离的线下店铺优惠券领取数')
figure = go.Figure(data=trace, layout=layout)
#py.offline.plot(figure, filename='fig.html')
figure

In [253]:
temp = train_offline[train_offline['label']==1].groupby('Distance')['Distance'].count()
temp.index = temp.index.astype(int)
temp = temp.sort_index()

trace = go.Scatter(
        x=temp.index,
        y=temp.values)

layout = go.Layout(
        title='2016年上半年不同距离的线下店铺优惠券使用数')
figure = go.Figure(data=trace, layout=layout)
#py.offline.plot(figure, filename='fig.html')
figure

In [255]:
temp = train_offline[train_offline['label']==1].groupby('Distance')['Distance'].count()/train_offline[~train_offline['Date_received'].isna()].groupby('Distance')['Distance'].count()
temp.index = temp.index.astype(int)
temp = temp.sort_index()

trace = go.Scatter(
        x=temp.index,
        y=temp.values)

layout = go.Layout(
        title='2016年上半年不同距离的线下店铺优惠券使用率')
figure = go.Figure(data=trace, layout=layout)
#py.offline.plot(figure, filename='fig.html')
figure